## Credit
[F5-TTS](https://github.com/SWivid/F5-TTS) <br>
This unnecessary extra code is written by NeuralFalcon, but it works.

In [ ]:
#@title Install F5-TTS , Download Model and Restart Session
import locale
locale.getpreferredencoding = lambda: "UTF-8"
import os
import urllib.request
import urllib.error
from tqdm import tqdm
import shutil
def conditional_download(url, download_file_path, redownload=False):
    print(f"Downloading {os.path.basename(download_file_path)}")
    base_path = os.path.dirname(download_file_path)

    # Create the directory if it doesn't exist
    if not os.path.exists(base_path):
        os.makedirs(base_path)

    # Skip download if the file exists and redownload is False
    if os.path.exists(download_file_path) and not redownload:
        print(f"File {download_file_path} already exists. Skipping download.")
        return

    # If redownload is True, remove the existing file
    if os.path.exists(download_file_path) and redownload:
        os.remove(download_file_path)

    # Try opening the URL and get the total file size
    try:
        request = urllib.request.urlopen(url)
        total = int(request.headers.get('Content-Length', 0))
    except urllib.error.URLError as e:
        print(f"Error: Unable to open the URL - {url}")
        print(f"Reason: {e.reason}")
        return

    # Start downloading with a progress bar
    with tqdm(total=total, desc=f"Downloading {os.path.basename(download_file_path)}", unit='B', unit_scale=True, unit_divisor=1024) as progress:
        try:
            urllib.request.urlretrieve(url, download_file_path, reporthook=lambda count, block_size, total_size: progress.update(block_size))
        except urllib.error.URLError as e:
            print(f"Error: Failed to download the file from the URL - {url}")
            print(f"Reason: {e.reason}")
            return

    print(f"Download successful! Saved at: {download_file_path}")


def download_models(base_path, redownload=False):
    print("Starting model downloads...")
    model_urls = [
        ("https://huggingface.co/SWivid/E2-TTS/resolve/main/E2TTS_Base/model_1200000.pt", f"{base_path}/ckpts/E2TTS_Base/model_1200000.pt"),
        # ("https://huggingface.co/SWivid/E2-TTS/resolve/main/E2TTS_Base/model_1200000.safetensors", f"{base_path}/ckpts/E2TTS_Base/model_1200000.safetensors"),
        ("https://huggingface.co/SWivid/F5-TTS/resolve/main/F5TTS_Base/model_1200000.pt", f"{base_path}/ckpts/F5TTS_Base/model_1200000.pt"),
        # ("https://huggingface.co/SWivid/F5-TTS/resolve/main/F5TTS_Base/model_1200000.safetensors", f"{base_path}/ckpts/F5TTS_Base/model_1200000.safetensors"),
        ("https://huggingface.co/charactr/vocos-mel-24khz/resolve/main/pytorch_model.bin", f"{base_path}/ckpts/vocos-mel-24khz/pytorch_model.bin"),
        ("https://huggingface.co/charactr/vocos-mel-24khz/resolve/main/config.yaml", f"{base_path}/ckpts/vocos-mel-24khz/config.yaml")
    ]

    for url, path in model_urls:
        conditional_download(url, path, redownload=redownload)

    print("All models downloaded successfully.")

def download_whisper_model(base_path, redownload=False):
    print("Starting model downloads...")
    # https://huggingface.co/deepdml/faster-whisper-large-v3-turbo-ct2
    model_urls = [
        ("https://huggingface.co/deepdml/faster-whisper-large-v3-turbo-ct2/resolve/main/config.json", f"{base_path}/faster-whisper-large-v3-turbo-ct2/config.json"),
        ("https://huggingface.co/deepdml/faster-whisper-large-v3-turbo-ct2/resolve/main/model.bin", f"{base_path}/faster-whisper-large-v3-turbo-ct2/model.bin"),
        ("https://huggingface.co/deepdml/faster-whisper-large-v3-turbo-ct2/resolve/main/preprocessor_config.json", f"{base_path}/faster-whisper-large-v3-turbo-ct2/preprocessor_config.json"),
        ("https://huggingface.co/deepdml/faster-whisper-large-v3-turbo-ct2/resolve/main/tokenizer.json", f"{base_path}/faster-whisper-large-v3-turbo-ct2/tokenizer.json"),
        ("https://huggingface.co/deepdml/faster-whisper-large-v3-turbo-ct2/resolve/main/vocabulary.json", f"{base_path}/faster-whisper-large-v3-turbo-ct2/vocabulary.json"),
    ]

    for url, path in model_urls:
        conditional_download(url, path, redownload=redownload)

    print("All models downloaded successfully.")
# base_path = "."
base_path = "/content"
%cd $base_path
install_path=f"{base_path}/F5-TTS"
if os.path.exists(install_path):
  shutil.rmtree(install_path)

# !git clone https://github.com/SWivid/F5-TTS.git
# They are updating the original repo, and I need a backup.Othewise the colab Notebook useless
!git clone https://github.com/NeuralFalconYT/F5-TTS.git
%cd $install_path
#downlaod model
download_models(install_path, redownload=False)
download_whisper_model(base_path, redownload=True)
#install packages
!pip install -r requirements.txt
!pip install pydub==0.25.1
!pip install pysrt==1.1.2
!pip install faster-whisper
!pip install ctranslate2==4.4.0
from IPython.display import clear_output
clear_output()
import time
time.sleep(5)
import os
os.kill(os.getpid(), 9)

After auto-restarting the session, run from the next cell.

In [1]:
# base_path = "."
base_path = "/content"

In [2]:
#@title import Model
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"
import os
install_path=f"{base_path}/F5-TTS"
os.chdir(install_path)
import os
import re
import torch
import torchaudio
from einops import rearrange
from vocos import Vocos
from model import CFM, UNetT, DiT
from model.utils import (
    load_checkpoint,
    get_tokenizer,
    convert_char_to_pinyin,
    save_spectrogram,
)
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import numpy as np
import librosa
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from pydub import AudioSegment
import re
import uuid
from tqdm.notebook import tqdm
import shutil
from IPython.display import clear_output
import gc
import time
import subprocess
from IPython.display import Audio

def is_gpu_memory_over_limit(limit_gb=14.5):
    # Run nvidia-smi and capture the output
    result = subprocess.run(['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader'],
                            stdout=subprocess.PIPE, text=True)

    # Split the result into lines (for each GPU if there are multiple)
    memory_used_mb_list = result.stdout.strip().splitlines()

    # Convert memory used from MB to GB and check each GPU's memory usage
    for i, memory_used_mb in enumerate(memory_used_mb_list):
        memory_used_gb = int(memory_used_mb) / 1024.0
        # print(f"GPU {i}: Current memory allocated: {memory_used_gb:.2f} GB")
        if memory_used_gb > limit_gb:
            # print(f"GPU {i} memory usage exceeds {limit_gb} GB.")
            return True

    # print("GPU memory usage is within safe limits.")
    return False



# Load Whisper model
def load_whisper():
    global whisper_pipe,whisper_model
    try:
        if whisper_pipe is not None:
          del whisper_pipe
          whisper_pipe=None
        if whisper_model is not None:
          del whisper_model
          whisper_model=None
        gc.collect()
        torch.cuda.empty_cache()
        # print("Free GPU memeory")
        time.sleep(2)
    except:
      pass
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

    model_id = "openai/whisper-large-v3-turbo"

    whisper_model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
    )
    whisper_model.to(device)

    processor = AutoProcessor.from_pretrained(model_id)

    whisper_pipe = pipeline(
        "automatic-speech-recognition",
        model=whisper_model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        torch_dtype=torch_dtype,
        device=device,
    )
    return whisper_pipe,whisper_model

# Initialize vocoder and model function
def initialize_vocoder_and_model(
    exp_name="F5TTS_Base",
    ckpt_step=1200000,
    device="cuda",
    target_sample_rate=24000,
    n_mel_channels=100,
    hop_length=256,
    dataset_name="Emilia_ZH_EN",
    tokenizer="pinyin",
    ode_method='euler',
    use_ema=True
):
    global vocos,model
    try:
      if vocos is not None:
        del vocos
        vocos=None
      if model is not None:
        del model
        model=None
      gc.collect()
      torch.cuda.empty_cache()
      # print("Free GPU memeory")
      time.sleep(2)
    except:
      pass
    # Set model configuration based on experiment name
    if exp_name == "F5TTS_Base":
        model_cls = DiT
        model_cfg = dict(dim=1024, depth=22, heads=16, ff_mult=2, text_dim=512, conv_layers=4)
    elif exp_name == "E2TTS_Base":
        model_cls = UNetT
        model_cfg = dict(dim=1024, depth=24, heads=16, ff_mult=4)

    # Initialize vocoder
    vocos_local_path = "./ckpts/vocos-mel-24khz"
    vocos = Vocos.from_hparams(f"{vocos_local_path}/config.yaml")
    state_dict = torch.load(f"{vocos_local_path}/pytorch_model.bin", map_location=device)
    vocos.load_state_dict(state_dict)
    vocos.eval()

    # Initialize tokenizer
    vocab_char_map, vocab_size = get_tokenizer(dataset_name, tokenizer)

    # Initialize model
    model = CFM(
        transformer=model_cls(
            **model_cfg,
            text_num_embeds=vocab_size,
            mel_dim=n_mel_channels
        ),
        mel_spec_kwargs=dict(
            target_sample_rate=target_sample_rate,
            n_mel_channels=n_mel_channels,
            hop_length=hop_length,
        ),
        odeint_kwargs=dict(
            method=ode_method,
        ),
        vocab_char_map=vocab_char_map,
    ).to(device)

    # Load the model checkpoint
    ckpt_path = f"ckpts/{exp_name}/model_{ckpt_step}.pt"
    model = load_checkpoint(model, ckpt_path, device, use_ema=use_ema)

    return vocos, model

def merge_audio(audio_list, save_path):
    # Initialize an empty audio segment
    merged_audio = AudioSegment.empty()

    # Loop through the list of audio files
    for audio_file in audio_list:
        # Load each audio file
        audio_segment = AudioSegment.from_wav(audio_file)
        # Append to the merged audio segment
        merged_audio += audio_segment

    # Export the merged audio to the specified save path
    merged_audio.export(save_path, format="wav")

def chunks_sentences(paragraph, join_limit=2):
    sentences = sent_tokenize(paragraph)
    # Initialize an empty list to store the new sentences
    new_sentences = []

    # Iterate through the list of sentences in steps of 'join_limit'
    for i in range(0, len(sentences), join_limit):
        # Join the sentences with a space between them
        new_sentence = ' '.join(sentences[i:i + join_limit])
        new_sentences.append(new_sentence)
    return new_sentences


def clean_file_name(file_path):
    # Get the base file name and extension
    file_name = os.path.basename(file_path)
    file_name, file_extension = os.path.splitext(file_name)

    # Replace non-alphanumeric characters with an underscore
    cleaned = re.sub(r'[^a-zA-Z\d]+', '_', file_name)

    # Remove any multiple underscores
    clean_file_name = re.sub(r'_+', '_', cleaned).strip('_')

    # Generate a random UUID for uniqueness
    random_uuid = uuid.uuid4().hex[:6]

    # Combine cleaned file name with the original extension
    clean_file_path = os.path.join(os.path.dirname(file_path), clean_file_name + f"_{random_uuid}" + file_extension)

    return clean_file_path


def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"{base_path}/f5_Voice/{truncated_text}_{random_string}.wav"
    file_name=clean_file_name(file_name)
    return file_name



import os
from pydub import AudioSegment


def is_audio_duration_greater_than_30s(audio_path,max_duration=30):
    try:
        audio = AudioSegment.from_file(audio_path)
    except Exception as e:
        print(f"Error loading audio file: {e}")
        return False

    # Get the duration in seconds
    duration = len(audio) / 1000  # pydub works in milliseconds

    # Check if the duration is greater than 30 seconds
    return duration > max_duration

def trim_audio(input_audio_path,max_duration=30):
    # Create output folder if it doesn't exist
    global base_path
    output_folder=f"{base_path}/trim_audio"
    os.makedirs(output_folder, exist_ok=True)

    # Load the audio file
    audio = AudioSegment.from_file(input_audio_path)

    # Check the duration of the audio in seconds
    duration = len(audio) / 1000  # pydub works in milliseconds

    # Trim the audio if it exceeds the max_duration
    if duration > max_duration:
        trimmed_audio = audio[:max_duration * 1000]  # Trim to max_duration
    else:
        trimmed_audio = audio

    # Generate a new file name
    base_name = os.path.splitext(os.path.basename(input_audio_path))[0]
    output_file = f"{output_folder}/{base_name}_trimmed.wav"
    # output_file=clean_file_name(output_file)
    trimmed_audio.export(output_file, format="wav")

    return output_file


def process_audio(reference_audio, max_duration=15):
    global old_trim_audio,base_path

    # Check if the audio duration exceeds max_duration
    if is_audio_duration_greater_than_30s(reference_audio, max_duration):
        f_base_name = os.path.basename(reference_audio)
        f_name, f_extension = os.path.splitext(f_base_name)
        trimmed_audio_path = f"{base_path}/trim_audio/{f_name}_trimmed.wav"

        # Check if we've already trimmed this audio
        if old_trim_audio == trimmed_audio_path:
            reference_audio = trimmed_audio_path  # Use existing trimmed audio
            # print("skipping because same trim audio")
        else:
            reference_audio = trim_audio(reference_audio, max_duration)  # Trim the audio
            old_trim_audio = reference_audio  # Update the old trimmed audio path

    return reference_audio


# Voice cloning function

def voice_clone(reference_audio, text, output_dir="", target_sample_rate=24000,remove_silence = False,fix_duration=None,chunks=0,exp_name="F5TTS_Base",progress_bar=True):
    global device,old_audio_path,old_ref_text,old_exp_name
    global whisper_pipe,whisper_model,vocos,model
    global seed
    reference_audio=process_audio(reference_audio, max_duration=15)

    if old_exp_name==exp_name:
      pass
    else:
      vocos, model = initialize_vocoder_and_model(device=device,exp_name=exp_name)
      old_exp_name=exp_name
    if is_gpu_memory_over_limit(limit_gb=14.5):
      whisper_pipe,whisper_model = load_whisper()
      device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
      vocos, model = initialize_vocoder_and_model(device=device,exp_name=exp_name)
    # clear_output()
    # seed = None
    final_audio_path= tts_file_name(text)
    output_dir=f"{base_path}/f5_Voice/temp"
    nfe_step = 32
    cfg_strength = 2.
    ode_method = 'euler'  # euler | midpoint
    speed = 1.
    # target_sample_rate = 24000
    # fix_duration=27
    target_rms = 0.1
    hop_length = 256  # Ensure hop_length is define
    tokenizer="pinyin"
    sway_sampling_coef = -1.

    fix_duration = fix_duration  # None (will linear estimate. if code-switched, consider fix) | float (total in seconds, include ref audio)

    # Get the reference audio text
    if old_audio_path==reference_audio:
      ref_text=old_ref_text
      # print("skipping because same audio file")
    else:
      ref_text = whisper_pipe(reference_audio)['text'].strip()
      old_audio_path=reference_audio
      old_ref_text=ref_text



    # Ensure output directory exists
    if os.path.exists(output_dir):
      shutil.rmtree(output_dir)
    os.makedirs(output_dir)

    # Load audio
    audio, sr = torchaudio.load(reference_audio)
    if audio.shape[0] > 1:
        audio = torch.mean(audio, dim=0, keepdim=True)

    # Normalize audio
    rms = torch.sqrt(torch.mean(torch.square(audio)))
    if rms < target_rms:
        audio = audio * target_rms / rms

    # Resample audio if necessary
    if sr != target_sample_rate:
        resampler = torchaudio.transforms.Resample(sr, target_sample_rate)
        audio = resampler(audio)
    audio = audio.to(device)

    if chunks==0:
      prompts=[text]
    else:
      prompts=chunks_sentences(text, join_limit=chunks)
    audio_list=[]

    number_of_prompts=len(prompts)
    iterable = tqdm(enumerate(prompts), total=len(prompts), desc="Processing Prompts") if progress_bar else enumerate(prompts)
    # for i,text in enumerate(prompts):
    # for i, text in tqdm(enumerate(prompts), total=len(prompts), desc="Processing Prompts"):
    for i, text in iterable:
      gen_text = text.strip()
      # Prepare text
      text_list = [ref_text + gen_text]
      if tokenizer == "pinyin":
          final_text_list = convert_char_to_pinyin(text_list)
      else:
          final_text_list = [text_list]
      # print(f"text  : {text_list}")
      # print(f"pinyin: {final_text_list}")

      # Calculate duration
      ref_audio_len = audio.shape[-1] // hop_length
      if fix_duration is not None:
          if number_of_prompts==1:
            duration = int(fix_duration * target_sample_rate / hop_length)
      else:
          zh_pause_punc = r"。，、；：？！"
          ref_text_len = len(ref_text) + len(re.findall(zh_pause_punc, ref_text))
          gen_text_len = len(gen_text) + len(re.findall(zh_pause_punc, gen_text))
          duration = ref_audio_len + int(ref_audio_len / ref_text_len * gen_text_len / speed)

      # Inference
      with torch.inference_mode():
          generated, _ = model.sample(
              cond=audio,
              text=final_text_list,
              duration=duration,
              steps=nfe_step,
              cfg_strength=cfg_strength,
              sway_sampling_coef=sway_sampling_coef,
              seed=seed,
          )

      generated = generated[:, ref_audio_len:, :]
      generated_mel_spec = rearrange(generated, '1 n d -> 1 d n')
      generated_wave = vocos.decode(generated_mel_spec.cpu())
      if rms < target_rms:
          generated_wave = generated_wave * rms / target_rms

      if isinstance(generated_wave, torch.Tensor):
          generated_wave = generated_wave.squeeze().cpu().numpy()

      # Remove silence

      if remove_silence:
          # Detect non-silent intervals
          non_silent_intervals = librosa.effects.split(generated_wave, top_db=30)

          # Concatenate non-silent parts
          non_silent_wave = np.array([])
          for interval in non_silent_intervals:
              start, end = interval
              non_silent_wave = np.concatenate([non_silent_wave, generated_wave[start:end]])

          # Replace generated_wave with the non-silent version
          generated_wave = non_silent_wave
          generated_wave_tensor = torch.tensor(generated_wave).unsqueeze(0)
      else:
          generated_wave_tensor = torch.tensor(generated_wave).unsqueeze(0)

      # Ensure that generated_wave_tensor is 2D (batch size, channels)
      if len(generated_wave_tensor.shape) == 1:
          generated_wave_tensor = generated_wave_tensor.unsqueeze(0)

      # Save the generated audio
      save_audio_path = f"{output_dir}/{i}.wav"
      torchaudio.save(save_audio_path, generated_wave_tensor, target_sample_rate)
      audio_list.append(save_audio_path)
    if len(audio_list)==1:
      shutil.copy(audio_list[-1],final_audio_path)
    elif len(audio_list)>1:
      merge_audio(audio_list, final_audio_path)
    else:
      final_audio_path=None
    return final_audio_path

whisper_pipe = None
whisper_model=None
vocos = None
model = None
seed = None
whisper_pipe,whisper_model = load_whisper()
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
vocos, model = initialize_vocoder_and_model(device=device,exp_name="F5TTS_Base")
old_audio_path=""
old_ref_text=""
old_trim_audio=""
old_exp_name="F5TTS_Base"
os.makedirs(f"{base_path}/f5_Voice", exist_ok=True)
clear_output()
print("Model Import Complete")

Model Import Complete


In [3]:
#@title Uplaod Reference Audio File
import os
from google.colab import files
from IPython.display import clear_output

def upload_audio():
    upload_folder = f"{base_path}/user_upload"

    # Ensure the upload folder exists
    os.makedirs(upload_folder, exist_ok=True)

    # Change to the upload directory
    os.chdir(upload_folder)

    # Upload the files
    uploaded = files.upload()

    # Switch back to the original install directory (ensure install_path is defined)
    os.chdir(install_path)

    audio_name_list = []

    for fn in uploaded.keys():
        file_path = f"{upload_folder}/{fn}"
        save_file_path = clean_file_name(file_path)

        # Rename the file with cleaned file name
        os.rename(file_path, save_file_path)
        audio_name_list.append(save_file_path)

    # Filter audio files based on valid audio extensions
    audio_extensions = ('.wav', '.mp3')
    valid_audio_files = [f for f in audio_name_list if f.lower().endswith(audio_extensions)]

    # Clear the output (for Google Colab)
    clear_output()

    if valid_audio_files:
        # Return the first valid audio file found
        return valid_audio_files[0]
    else:
        # Print message if no valid audio files were uploaded
        print("Please upload an audio file.")
        return None
upload_audio()

'/content/user_upload/RD_Playful_ChildlikeVocal_3_AudioTrimmer_com_237097.wav'

In [8]:
#@title Generate TTS
Reference_Audio_Path= '/content/user_upload/RD_Playful_ChildlikeVocal_3_AudioTrimmer_com_237097.wav'  # @param {type: "string"}
TTS_Text = 'Sinners... Radio Demon intercepts a broadcast from hell. still alive A killer who mailed death threats, coded in arrogance. A case cold as a crypt... but the signal’ssss... still alive. The 340 cipher? Solved in 2020 and the devil is listening. '  # @param {type: "string"}
Remove_Silence = True  # @param {type: "boolean"}
# Split_Sentences = 0  # @param {type: "number"}
if len(TTS_Text)<=300:
  Split_Sentences=0
if len(TTS_Text)>300:
  Split_Sentences=3
Choose_Model = "F5TTS_Base" # @param ['F5TTS_Base', 'E2TTS_Base']
seed = None
cloned_voice_path=voice_clone(Reference_Audio_Path, TTS_Text,remove_silence = Remove_Silence,chunks=Split_Sentences,exp_name=Choose_Model)
clear_output()
print(f"TTS Save at {cloned_voice_path}")
Audio(cloned_voice_path)

TTS Save at /content/f5_Voice/sinners_radio_demon_in_997D5014_daa0ff.wav


In [ ]:
#@title Using Gradio Interface
import gradio as gr
def gradio_call(Reference_Audio_Path,TTS_Text,Remove_Silence,Choose_Model):
  global seed
  seed = None
  if len(TTS_Text)<=300:
    Split_Sentences=0
  if len(TTS_Text)>300:
    Split_Sentences=3
  cloned_voice_path=voice_clone(Reference_Audio_Path, TTS_Text,remove_silence = Remove_Silence,chunks=Split_Sentences,exp_name=Choose_Model)
  return cloned_voice_path

demo_examples = [["/content/F5-TTS/tests/ref_audio/test_en_1_ref_short.wav","Hi How are you",True,"F5TTS_Base"]]
gradio_inputs=[gr.Audio(label="Reference Audio", type="filepath"),
                       gr.Textbox(label="Input Text"),
                       gr.Checkbox(value=True,label="Remove Silence From Generated Audio"),
                       gr.Dropdown(label="Choose TTS Model",choices=['F5TTS_Base', 'E2TTS_Base'],value="F5TTS_Base")
                       ]
gradio_outputs=[gr.Audio(label="Generated Audio")]
demo = gr.Interface(fn=gradio_call, inputs=gradio_inputs,outputs=gradio_outputs , title="F5-TTS Demo",examples=demo_examples)
demo.launch(debug=True,share=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().


/usr/local/lib/python3.10/dist-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.1, however version 5.0.1 is available, please upgrade. 
--------
  warnings.warn(


Running on public URL: https://49415eb7fa5395c3ce.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Processing Prompts:   0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-2-a0bba19cbe2a>:134: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f"{vocos_local_path}/pytorch_model.bin", map_location=device)
/cont

Processing Prompts:   0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-2-a0bba19cbe2a>:134: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f"{vocos_local_path}/pytorch_model.bin", map_location=device)
/cont

Processing Prompts:   0%|          | 0/1 [00:00<?, ?it/s]

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://49415eb7fa5395c3ce.gradio.live


# Subtitle Dubbing

In [ ]:
#@title Mount Google Drive to upload larger video or audio and autosave the generated SRTDubbing Audio
from google.colab import drive
import os
Mount_Drive = False  # @param {type: "boolean"}
if Mount_Drive:
  drive.mount('/content/gdrive')

In [ ]:
#@title load faster-whisper
import math
import torch
import gc
import time
import subprocess
from IPython.display import Audio
from faster_whisper import WhisperModel
import os
import mimetypes
import shutil
import re
import uuid
import locale
locale.getpreferredencoding = lambda: "UTF-8"
def clean_file_name(file_path):
    # Get the base file name and extension
    file_name = os.path.basename(file_path)
    file_name, file_extension = os.path.splitext(file_name)

    # Replace non-alphanumeric characters with an underscore
    cleaned = re.sub(r'[^a-zA-Z\d]+', '_', file_name)

    # Remove any multiple underscores
    clean_file_name = re.sub(r'_+', '_', cleaned).strip('_')

    # Generate a random UUID for uniqueness
    random_uuid = uuid.uuid4().hex[:6]

    # Combine cleaned file name with the original extension
    clean_file_path = os.path.join(os.path.dirname(file_path), clean_file_name + f"_{random_uuid}" + file_extension)

    return clean_file_path

def get_audio_file(uploaded_file):
    global base_path
    # ,device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    # Detect the file type (audio/video)
    mime_type, _ = mimetypes.guess_type(uploaded_file)
    # Create the folder path to store audio files
    audio_folder = f"{base_path}/subtitle_audio"
    os.makedirs(audio_folder, exist_ok=True)
    # Initialize variable for the audio file path
    audio_file_path = ""
    if mime_type and mime_type.startswith('audio'):
        # If it's an audio file, save it as is
        audio_file_path = os.path.join(audio_folder, os.path.basename(uploaded_file))
        audio_file_path=clean_file_name(audio_file_path)
        shutil.copy(uploaded_file, audio_file_path)  # Move file to audio folder

    elif mime_type and mime_type.startswith('video'):
        # If it's a video file, extract the audio
        audio_file_name = os.path.splitext(os.path.basename(uploaded_file))[0] + ".mp3"
        audio_file_path = os.path.join(audio_folder, audio_file_name)
        audio_file_path=clean_file_name(audio_file_path)

        # Extract the file extension from the uploaded file
        file_extension = os.path.splitext(uploaded_file)[1]  # Includes the dot, e.g., '.mp4'

        # Generate a random UUID and create a new file name with the same extension
        random_uuid = uuid.uuid4().hex[:6]
        new_file_name = random_uuid + file_extension

        # Set the new file path in the subtitle_audio folder
        new_file_path = os.path.join(audio_folder, new_file_name)

        # Copy the original video file to the new location with the new name
        shutil.copy(uploaded_file, new_file_path)
        if device=="cuda":
          command = f"ffmpeg -hwaccel cuda -i {new_file_path} {audio_file_path} -y"
        else:
          command = f"ffmpeg -i {new_file_path} {audio_file_path} -y"

        # if device=="cuda":
        #   command = f"ffmpeg -hwaccel cuda -i {new_file_path} -vn -ab 320k -ar 48000 -c:a copy -y {audio_file_path}"
        # else:
        #   command = f"ffmpeg -i {new_file_path} -vn -ab 320k -ar 48000 -y {audio_file_path} -y"

        subprocess.run(command, shell=True)
        if os.path.exists(new_file_path):
          os.remove(new_file_path)
    # Return the saved audio file path
    return audio_file_path


def is_gpu_memory_over_limit(limit_gb=14.5):
    # Run nvidia-smi and capture the output
    result = subprocess.run(['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader'],
                            stdout=subprocess.PIPE, text=True)

    # Split the result into lines (for each GPU if there are multiple)
    memory_used_mb_list = result.stdout.strip().splitlines()

    # Convert memory used from MB to GB and check each GPU's memory usage
    for i, memory_used_mb in enumerate(memory_used_mb_list):
        memory_used_gb = int(memory_used_mb) / 1024.0
        # print(f"GPU {i}: Current memory allocated: {memory_used_gb:.2f} GB")
        if memory_used_gb > limit_gb:
            # print(f"GPU {i} memory usage exceeds {limit_gb} GB.")
            return True

    # print("GPU memory usage is within safe limits.")
    return False

def convert_seconds_to_hms(seconds):
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    milliseconds = math.floor((seconds % 1) * 1000)
    output = f"{int(hours):02}:{int(minutes):02}:{int(seconds):02},{milliseconds:03}"
    return output


def load_whisper_turbo_model():
  global whisper_turbo_model,base_path
  try:
    if whisper_turbo_model is not None:
      del whisper_turbo_model
      whisper_turbo_model=None
    gc.collect()
    torch.cuda.empty_cache()
    time.sleep(2)
  except:
      pass
  # model_name="faster-whisper-large-v3-turbo-ct2"
  model_name=f"{base_path}/faster-whisper-large-v3-turbo-ct2"
  device = "cuda" if torch.cuda.is_available() else "cpu"
  if device == "cuda":
      try:
          whisper_turbo_model = WhisperModel(model_name, device="cuda", compute_type="float16")
      except Exception as e:
          whisper_turbo_model = WhisperModel(model_name, device="cuda", compute_type="int8_float16")
  else:
      whisper_turbo_model = WhisperModel(model_name, device="cpu", compute_type="int8")
  return whisper_turbo_model

def subtitle_maker(input_file):
  global base_path,whisper_turbo_model
  if is_gpu_memory_over_limit(limit_gb=14.5):
    whisper_turbo_model=load_whisper_turbo_model()
  base_name = os.path.splitext(os.path.basename(input_file))[0]
  random_uuid = uuid.uuid4().hex[:4]
  subtitle_folder = f"{base_path}/Generated_Subtitle"
  os.makedirs(subtitle_folder, exist_ok=True)
  srt_file_name =f"{subtitle_folder}/{base_name}.srt"
  srt_file_name=clean_file_name(srt_file_name)
  audio_path=get_audio_file(input_file)
  segments, info = whisper_turbo_model.transcribe(audio_path, beam_size=5,vad_filter=True,vad_parameters=dict(min_silence_duration_ms=500))
  saved_segments = list(segments)
  count = 0
  sts=""
  with open(srt_file_name, 'w',encoding="utf-8") as f:
    for i in saved_segments:
        segment=list(i)
        id=segment[0]
        seek=segment[1]
        start=segment[2]
        end=segment[3]
        text=segment[4]
        sts+=str(text)
        count +=1
        duration = f"{convert_seconds_to_hms(start)} --> {convert_seconds_to_hms(end)}\n"
        text = f"{text.lstrip()}\n\n"
        f.write(f"{count}\n{duration}{text}")
  sts=sts.strip()
  text_path=srt_file_name.replace(".srt",".txt")
  with open(text_path, 'w') as file:
      file.write(sts)
  if os.path.exists(audio_path):
    os.remove(audio_path)
  # Check if Google Drive is mounted
  if os.path.exists("/content/gdrive/MyDrive/"):
      # Define the target drive folder
      drive_folder = "/content/gdrive/MyDrive/whisper_subtitle"
      # Create the target folder if it doesn't exist
      os.makedirs(drive_folder, exist_ok=True)
      # Get the base name of the SRT file
      file_name = os.path.basename(srt_file_name)
      # Copy the SRT file to the drive folder
      shutil.copy(srt_file_name, f"{drive_folder}/{file_name}")
      # Copy the text file to the drive folder with the new name
      text_file_name = file_name.replace(".srt", ".txt")
      shutil.copy(text_path, f"{drive_folder}/{text_file_name}")
      # Print confirmation message
      print(f"Copied to drive: '{drive_folder}/{file_name}'")
      # print(f"Copied to drive: '{drive_folder}/{text_file_name}'")
  return str(srt_file_name),str(text_path),sts
base_path="/content"
# base_path="."
whisper_turbo_model=None
whisper_turbo_model=load_whisper_turbo_model()

In [ ]:
#@title Upload Audio or Video to Generate Subtitle
import os
from google.colab import files
from IPython.display import clear_output

def upload_local_video_or_audio():
    global base_path
    upload_folder = f"{base_path}/uploaded_audio_video"
    # Ensure the upload folder exists
    os.makedirs(upload_folder, exist_ok=True)
    # Change to the upload directory
    os.chdir(upload_folder)
    # Upload the files
    uploaded = files.upload()
    # Switch back to the original install directory (ensure install_path is defined)
    os.chdir(install_path)
    # Create a list to store the paths of uploaded files
    upload_list = []
    # Append each uploaded file to the list
    for fn in uploaded.keys():
        upload_list.append(f"{upload_folder}/{fn}")
    # Clear the output (for Google Colab)
    clear_output()
    # Return the last uploaded file (assuming that's what you want)
    return upload_list[-1] if upload_list else None

# Call the function
upload_local_video_or_audio()


'/content/uploaded_audio_video/youtube.mp3'

In [ ]:
Audio_Or_Video_File_Path = '/content/uploaded_audio_video/youtube.mp3'  # @param {type: "string"}
srt_file_name,text_file_name,text_=subtitle_maker(Audio_Or_Video_File_Path)
srt_file_name

'/content/Generated_Subtitle/youtube_0390d2.srt'

In [ ]:
#@title Generate Audio File From Subtitle
%cd $install_path
def your_tts(text,audio_path,language):
  global Reference_Audio_File,Clone_Method,Seed,Remove_Silence_From_TTS
  if len(text)<=300:
    Split_Sentences=0
  if len(text)>300:
    Split_Sentences=3
  cloned_voice_path=voice_clone(Reference_Audio_File, text,remove_silence = Remove_Silence_From_TTS,chunks=Split_Sentences,exp_name=Clone_Method,progress_bar=False)
  shutil.copy(cloned_voice_path,audio_path)



from tqdm.notebook import tqdm

import os
def get_subtitle_Dub_path(srt_file_path,Language):
  file_name = os.path.splitext(os.path.basename(srt_file_path))[0]
  if not os.path.exists("/content/TTS_DUB"):
    os.mkdir("/content/TTS_DUB")
  random_string = str(uuid.uuid4())[:6]
  new_path=f"/content/TTS_DUB/{file_name}_{random_string}.wav"
  return new_path






import subprocess
import json

def get_video_duration(video_path):
    try:
        # Run ffmpeg command to get video information in JSON format
        result = subprocess.run(
            ['ffmpeg', '-i', video_path, '-f', 'ffmetadata', '-'],
            stderr=subprocess.PIPE,
            text=True
        )

        # Parse the duration from the stderr output
        for line in result.stderr.split('\n'):
            if 'Duration' in line:
                duration_str = line.split('Duration: ')[1].split(',')[0]
                h, m, s = duration_str.split(':')
                duration = int(h) * 3600 + int(m) * 60 + float(s)
                return duration
    except Exception as e:
        print(f"Error: {e}")
        return None


def replace_audio(video_path,audio_path):
  if not video_path.lower().endswith(".mp4"):
    return
  tts_audio = AudioSegment.from_file(dub_save_path)
  audio_duration = len(tts_audio)/1000
  video_duration=get_video_duration(video_path)
  slience_duration=video_duration-audio_duration
  audio_segment = AudioSegment.from_file(audio_path)
  slience_Segment= AudioSegment.silent(duration=slience_duration)
  marge_audio=audio_segment+slience_Segment
  marge_audio.export("/content/new_audio.wav", format="wav")
  command=f"ffmpeg -i {video_path}  -i /content/new_audio.wav -map 0:v -map 1:a -c:v copy -shortest /content/output.mp4 -y"
  var=os.system(command)
  if var==0:
    if os.path.exists("/content/gdrive/MyDrive/upload"):
      file_name = os.path.basename(video_path)
      shutil.copy("/content/output.mp4", f"/content/gdrive/MyDrive/upload/change_audio_{file_name}")
      print(f"Copied at drive '/content/gdrive/MyDrive/upload/change_audio_{file_name}'")
      return f"/content/gdrive/MyDrive/upload/change_audio_{file_name}"
  else:
    print(command)
    return None





import pysrt

def clean_srt(input_path):
    file_name = os.path.basename(input_path)
    output_folder = "/content/save_srt"
    if not os.path.exists(output_folder):
        os.mkdir(output_folder)
    output_path = f"{output_folder}/{file_name}"

    def clean_srt_line(text):
        bad_list = ["[", "]", "♫", "\n"]
        for i in bad_list:
            text = text.replace(i, "")
        return text.strip()

    # Load the subtitle file
    subs = pysrt.open(input_path)

    # Iterate through each subtitle and print its details
    with open(output_path, "w", encoding='utf-8') as file:
        for sub in subs:
            file.write(f"{sub.index}\n")
            file.write(f"{sub.start} --> {sub.end}\n")
            file.write(f"{clean_srt_line(sub.text)}\n")
            file.write("\n")
        file.close()
    # print(f"Clean SRT saved at: {output_path}")
    return output_path
# Example usage



###

from pydub import AudioSegment
import shutil
import subprocess
import os
import uuid
import re




# dub_save_path=get_subtitle_Dub_path(srt_file_path,Language)
import time

# def your_tts(text,audio_path,language):
#   global Reference_Audio_File,Language,Clone_Method,Seed
#   wav_file_path=convert_to_wav(Reference_Audio_File)
#   if Clone_Method=="3s Quick Clone":
#     prompt_speech_16k,prompt_text = postprocess(wav_file_path,Language,audio_to_text=True)
#   else:
#     prompt_speech_16k,prompt_text = postprocess(wav_file_path,Language,audio_to_text=False)
#   temp_path=voice_clone_for_subtitle(text,prompt_speech_16k,prompt_text,Language,"3s Quick Clone",Seed,"/content/clone_voice")
#   shutil.copy(temp_path,audio_path)

class SRTDubbing:
    def __init__(self):
        pass

    @staticmethod
    def text_to_speech(text, audio_path, language, actual_duration):
        tts_filename = "temp.wav"
        your_tts(text,tts_filename,language)
        # Check the duration of the generated TTS audio
        tts_audio = AudioSegment.from_file(tts_filename)
        tts_duration = len(tts_audio)

        if actual_duration == 0:
            # If actual duration is zero, use the original TTS audio without modifications
            shutil.move(tts_filename, audio_path)
            return

        # If TTS audio duration is longer than actual duration, speed up the audio
        if tts_duration > actual_duration:
            speedup_factor = tts_duration / actual_duration
            speedup_filename = "speedup_temp.wav"

            # Use ffmpeg to change audio speed
            subprocess.run([
                "ffmpeg",
                "-i", tts_filename,
                "-filter:a", f"atempo={speedup_factor}",
                speedup_filename
            ], check=True)

            # Replace the original TTS audio with the sped-up version
            shutil.move(speedup_filename, audio_path)
        elif tts_duration < actual_duration:
            # If TTS audio duration is less than actual duration, add silence to match the duration
            silence_gap = actual_duration - tts_duration
            silence = AudioSegment.silent(duration=int(silence_gap))
            new_audio = tts_audio + silence

            # Save the new audio with added silence
            new_audio.export(audio_path, format="wav")
        else:
            # If TTS audio duration is equal to actual duration, use the original TTS audio
            shutil.move(tts_filename, audio_path)

    @staticmethod
    def make_silence(pause_time, pause_save_path):
        silence = AudioSegment.silent(duration=pause_time)
        silence.export(pause_save_path, format="wav")
        return pause_save_path

    @staticmethod
    def create_folder_for_srt(srt_file_path):
        srt_base_name = os.path.splitext(os.path.basename(srt_file_path))[0]
        random_uuid = str(uuid.uuid4())[:4]
        dummy_folder_path = "/content/dummy"
        if not os.path.exists(dummy_folder_path):
            os.makedirs(dummy_folder_path)
        folder_path = os.path.join(dummy_folder_path, f"{srt_base_name}_{random_uuid}")
        os.makedirs(folder_path, exist_ok=True)
        return folder_path

    @staticmethod
    def concatenate_audio_files(audio_paths, output_path):
        concatenated_audio = AudioSegment.silent(duration=0)
        for audio_path in audio_paths:
            audio_segment = AudioSegment.from_file(audio_path)
            concatenated_audio += audio_segment
        concatenated_audio.export(output_path, format="wav")

    def srt_to_dub(self, srt_file_path, dub_save_path,language='en'):
        result = self.read_srt_file(srt_file_path)
        new_folder_path = self.create_folder_for_srt(srt_file_path)
        join_path = []
        for i in tqdm(result):
        # for i in result:
            text = i['text']
            actual_duration = i['end_time'] - i['start_time']
            pause_time = i['pause_time']
            slient_path = f"{new_folder_path}/{i['previous_pause']}"
            self.make_silence(pause_time, slient_path)
            join_path.append(slient_path)
            tts_path = f"{new_folder_path}/{i['audio_name']}"
            self.text_to_speech(text, tts_path, language, actual_duration)
            join_path.append(tts_path)
        self.concatenate_audio_files(join_path, dub_save_path)

    @staticmethod
    def convert_to_millisecond(time_str):
      if isinstance(time_str, str):
          hours, minutes, second_millisecond = time_str.split(':')
          seconds, milliseconds = second_millisecond.split(",")

          total_milliseconds = (
              int(hours) * 3600000 +
              int(minutes) * 60000 +
              int(seconds) * 1000 +
              int(milliseconds)
          )

          return total_milliseconds
    @staticmethod
    def read_srt_file(file_path):
        entries = []
        default_start = 0
        previous_end_time = default_start
        entry_number = 1
        audio_name_template = "{}.wav"
        previous_pause_template = "{}_before_pause.wav"

        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()
            # print(lines)
            for i in range(0, len(lines), 4):
                time_info = re.findall(r'(\d+:\d+:\d+,\d+) --> (\d+:\d+:\d+,\d+)', lines[i + 1])
                start_time = SRTDubbing.convert_to_millisecond(time_info[0][0])
                end_time = SRTDubbing.convert_to_millisecond(time_info[0][1])

                current_entry = {
                    'entry_number': entry_number,
                    'start_time': start_time,
                    'end_time': end_time,
                    'text': lines[i + 2].strip(),
                    'pause_time': start_time - previous_end_time if entry_number != 1 else start_time - default_start,
                    'audio_name': audio_name_template.format(entry_number),
                    'previous_pause': previous_pause_template.format(entry_number),
                }

                entries.append(current_entry)
                previous_end_time = end_time
                entry_number += 1

        return entries

# Example usage
Enter_srt_file_path = '/content/Generated_Subtitle/youtube_0390d2.srt'  # @param {type: "string"}
srt_file_path=clean_srt(Enter_srt_file_path)

Reference_Audio_File="/content/F5-TTS/tests/ref_audio/test_en_1_ref_short.wav"# @param {type: "string"}
Remove_Silence_From_TTS = True  # @param {type: "boolean"}

# Language = "English" # @param [ 'English','Chinese', 'Japanese', 'Cantonese', 'Korean'] {allow-input: true}
Language = "English"
Clone_Method = "F5TTS_Base" # @param ['F5TTS_Base', 'E2TTS_Base'] {allow-input: true}
Seed=100000 # @param {type: "number"}
seed = Seed

srt_dubbing = SRTDubbing()
current_language=Language
dub_save_path=get_subtitle_Dub_path(srt_file_path,Language)
srt_dubbing.srt_to_dub(srt_file_path, dub_save_path,current_language)
from IPython.display import clear_output
clear_output()
print(f"{Language} Dub Audio File Save At : {dub_save_path}")
if os.path.exists("/content/gdrive/MyDrive/"):
  drive_folder = "/content/gdrive/MyDrive/srt_dub"
  os.makedirs(drive_folder, exist_ok=True)
  file_name = os.path.basename(dub_save_path)
  shutil.copy(dub_save_path, f"{drive_folder}/{file_name}")
  print(f"Copied at drive '{drive_folder}/{file_name}'")


# replace_audio_path=replace_audio(audio_file_path,dub_save_path)

from google.colab import files

files.download(dub_save_path)
# files.download(replace_audio_path)




English Dub Audio File Save At : /content/TTS_DUB/youtube_0390d2_5cf27a.wav


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>